In [9]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define NOT_EXIST -1

// 字典结构
typedef struct{
    char** seq;
    int* code;
    int size;
    int max_size;
}Dictionary;

//插入字典
void insert_seq(Dictionary* dict, char* seq){
    int i = dict->size;
    // 分配字符串空间
    dict->seq[i] = malloc(sizeof(char) * strlen(seq) + 1);
    dict->code[i] = i;
    dict->size++;
    // 将字符串加入分配的空间
    strcpy(dict->seq[i], seq);
}

void init_dictionary(Dictionary* dict, int max_size){
    dict->max_size = max_size;
    dict->size = 0;
    // 分配初始字符串数组空间
    dict->seq = malloc(sizeof(char *) * max_size);
    dict->code = malloc(sizeof(int) * max_size);
    insert_seq(dict, "#");
    // 初始字符串
    char seq[2] = "a";
    //插入26个字母至索引表 
    for(int i = 0;i < 26;i++){
        insert_seq(dict,seq);
        seq[0]++;
    }
}

int get_seq_code(Dictionary* dict,char* seq){
    // 顺序查找
    // todo:也许以后可以改进为二分查找，或者其他查找方式，这里为了简单就顺序查找咯
    for(int i=0;i<dict->size;i++){
        if(strcmp(dict->seq[i], seq) == 0){
            return dict->code[i];
        }
    }
    return NOT_EXIST;
}

char* get_code_seq(Dictionary* dict, int code){
    if (code < 0 || code >= dict->size){
        return NULL;
    }else{
        return dict->seq[code];
    }
    
}

void lzw_encode(char* text, Dictionary* dict){
    FILE* pFile;
    char current[1000];
    char next;
    int code;
    int i = 0;
//     if((pFile = fopen ("./codes", "wb"))==NULL)
//     {
//         printf("cant open the file");
//         exit(0);
//     }
    while(i < strlen(text)){
        sprintf(current, "%c", text[i]);
        next = text[i+1];
//         if(next=='\0'){
//            return;
//         }
        while(get_seq_code(dict,current) != NOT_EXIST){
            sprintf(current,"%s%c",current,next);
            i++;
            next = text[i+1];
            
        }  
        //多了一个next字符，需要抹掉
        current[strlen(current)-1] = '\0';
        //倒回一个位
        next = text[i]; 
        code = get_seq_code(dict,current);
        // 插入新的字符组合
        sprintf(current,"%s%c",current,next);
        insert_seq(dict,current);
        printf("%d",code);
        //fwrite(&code,sizeof(int),1,pFile);
    }
//     fclose (pFile);
//     free(pFile);
}


void lzw_decode(int codes[],int n,Dictionary* dict){
    int code;
    char pre[1000];
    char* output;
    code = codes[0];
    output = get_code_seq(dict,code);
    int i;
    printf("%s",output);
    for(i=1;i<n;i++){
        code = codes[i];
        strcpy(pre,output);
        output = get_code_seq(dict,code);
        sprintf(pre,"%s%c",pre,output[0]);
        insert_seq(dict,pre);
        printf("%s",output);
    }
}

void print_dictionart(Dictionary* dict){
    printf("===========================\n");
    printf("  Code            Sequence\n");
    printf("===========================\n");
    for(int i=0;i<dict->size;i++){
        printf("%4d%16c",dict->code[i],' ');
        printf("%s\n",dict->seq[i]);
    }
    printf("===========================\n");
}

int* read_code(char* file){
    FILE * fp;
    // TO-DO
    int buffer[1000];
    if((fp=fopen("codes","rb"))==NULL)
    {
        printf("cant open the file");
        exit(0);
    }
    fread(buffer,sizeof(int),1000,fp);
    int i =0;
    for(;i<1000;i++){
        printf("%d",buffer[i]);
    }
    return NULL;
}

int main()
{
    Dictionary dict;
    Dictionary dict_decode;
    init_dictionary(&dict, 1000);
    init_dictionary(&dict_decode,1000);
//     print_dictionart(&dict);
//     printf("%d",get_seq_code(&dict,"D"));
    char * text = "zhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaowei";
    printf("原始文本，文本长度 %d\n",strlen(text));
    printf("%s\n",text);
    int codes[] = {26,8,15,14,7,8,1,15,23,5,9,26,21,9,19,8,21,1,37,28,30,32,34,36,26,1,39,41,43,45,29,31,33,35,56,47,59,50,53,42,44,27,57,48,60,51,65,55,68,62,49,61,58,77,38,40,66,78,70,50,52,82,74,46,79,71,90,85,37,73,67,93,63,37,87,54,97,69,99};
    int n = (sizeof(codes)/sizeof(codes[0]));
    printf("压缩编码，编码长度 %d\n",n);
    lzw_encode(text,&dict);
    //print_dictionart(&dict);
    float radio = 1-1.0*n/(strlen(text));
    printf("\n解压解码，压缩比率 %.2f\%\n",radio*100);
    lzw_decode(codes,n,&dict_decode);
    return 0;
}

原始文本，文本长度 200
zhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaowei
压缩编码，编码长度 79
2681514781152359262191982113728303234362613941434529313335564759505342442757486051655568624961587738406678705052827446797190853773679363378754976999
解压解码，压缩比率 60.50%
zhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaoweizhonghaoweizuishuaizhonghaoweizauishuaizhonghaowei